In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv(r'C:\Users\agile\Downloads\Week3_GA_dataset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      748 non-null    object 
 1   V2      748 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB


In [6]:
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [23]:
df["V2"].unique()

array(['50.0', '13.0', '?', '20.0', '24.0', '12.0', '9.0', '46.0', '3.0',
       '10.0', '6.0', '5.0', '14.0', '11.0', '8.0', '16.0', '7.0', '2.0',
       '19.0', '4.0', '17.0', '1.0', '15.0', '22.0', '18.0', '38.0',
       '43.0', '34.0', '44.0', '26.0', '41.0', '21.0', '33.0'],
      dtype=object)

In [28]:
df["V2"].value_counts()

V2
1.0     158
2.0     112
3.0      87
5.0      62
4.0      61
6.0      52
7.0      42
8.0      31
9.0      24
11.0     22
10.0     14
12.0     14
14.0     13
16.0     12
13.0      9
15.0      5
?         5
17.0      4
24.0      3
22.0      2
19.0      2
20.0      2
21.0      2
34.0      1
41.0      1
26.0      1
44.0      1
50.0      1
43.0      1
38.0      1
18.0      1
46.0      1
33.0      1
Name: count, dtype: int64

In [30]:
value_counts_all = df.apply(pd.Series.value_counts)
print(value_counts_all)

            V1     V2  V3   V4     V5  Target
0.0        4.0    NaN NaN  NaN    NaN     NaN
1.0        7.0  158.0 NaN  NaN    NaN     NaN
10.0       3.0   14.0 NaN  NaN    NaN     NaN
11.0      84.0   22.0 NaN  NaN    NaN     NaN
12.0       6.0   14.0 NaN  NaN    NaN     NaN
...        ...    ...  ..  ...    ...     ...
13.0       NaN    NaN NaN  1.0    NaN     NaN
54.0       NaN    NaN NaN  1.0    NaN     NaN
NEGATIVE   NaN    NaN NaN  NaN  748.0     NaN
NO         NaN    NaN NaN  NaN    NaN   570.0
YES        NaN    NaN NaN  NaN    NaN   178.0

[155 rows x 6 columns]


In [31]:
from sklearn.impute import SimpleImputer

In [33]:
from sklearn.preprocessing import OrdinalEncoder,StandardScaler

In [34]:
from sklearn.feature_selection import VarianceThreshold

In [35]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [37]:
from sklearn.compose import ColumnTransformer

In [53]:
feature = [0,1,2,3]
predict = [4]

In [52]:
df = df.apply(pd.to_numeric, errors='coerce')

In [54]:
a = Pipeline(steps = [ 
('Imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

b = Pipeline(steps = [
    ('encoder', OrdinalEncoder())
])

In [55]:
processor = ColumnTransformer(
    transformers = [
  ('A',a,feature),('B',b,predict)  
])

In [56]:
full_pipeline = Pipeline(steps=[
    ('preprocessor',processor),
    ('selector', VarianceThreshold(threshold=0.1))
])

In [58]:
Ans = full_pipeline.fit_transform(df)
Ans.shape[1]

C:\Users\agile\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


4

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [85]:
df.replace('?', np.nan, inplace=True)

In [86]:
X = df.drop(columns='Target')

In [87]:
Y = df["Target"]

In [88]:
X['V1'] = pd.to_numeric(X['V1'], errors='coerce')
X['V2'] = pd.to_numeric(X['V2'], errors='coerce')
X['V3'] = pd.to_numeric(X['V3'], errors='coerce')
X['V4'] = pd.to_numeric(X['V4'], errors='coerce')
X['V5'] = X['V5'].astype(str)

In [89]:
X['V5'] = X['V5'].astype(str)

# Encode the target variable
Y = OrdinalEncoder().fit_transform(Y.values.reshape(-1, 1)).ravel()


In [90]:
preprocessor = ColumnTransformer(transformers=[
    ('imputer', SimpleImputer(strategy='mean'), ['V1', 'V2']),
    ('scaler', StandardScaler(), ['V1', 'V2', 'V3', 'V4']),
    ('encoder', OrdinalEncoder(), ['V5'])
])

# Full pipeline with logistic regression
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression())
])

In [91]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=42)
pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(Y_test,Y_pred)
accuracy

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [93]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data
df = pd.read_csv(r'C:\Users\agile\Downloads\Week3_GA_dataset.csv')
df.replace('?', np.nan, inplace=True)

# Separate features and target
X = df.drop(columns='Target')
y = df['Target']

# Convert appropriate columns
X['V1'] = pd.to_numeric(X['V1'], errors='coerce')
X['V2'] = pd.to_numeric(X['V2'], errors='coerce')
X['V3'] = pd.to_numeric(X['V3'], errors='coerce')
X['V4'] = pd.to_numeric(X['V4'], errors='coerce')
X['V5'] = X['V5'].astype(str)

# Encode target
y = OrdinalEncoder().fit_transform(y.values.reshape(-1, 1)).ravel()

# Updated numeric imputation (now includes all V1–V4)
numeric_features = ['V1', 'V2', 'V3', 'V4']
categorical_features = ['V5']

# Define preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    
    ('cat', OrdinalEncoder(), categorical_features)
])

# Complete pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression())
])

# Train/test split and model fitting
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.7540


In [94]:
from sklearn.feature_selection import RFE

In [95]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# Load data
df = pd.read_csv(r'C:\Users\agile\Downloads\Week3_GA_dataset.csv')
df.replace('?', np.nan, inplace=True)

# Separate features and target
X = df.drop(columns='Target')
y = df['Target']

# Convert types
X['V1'] = pd.to_numeric(X['V1'], errors='coerce')
X['V2'] = pd.to_numeric(X['V2'], errors='coerce')
X['V3'] = pd.to_numeric(X['V3'], errors='coerce')
X['V4'] = pd.to_numeric(X['V4'], errors='coerce')
X['V5'] = X['V5'].astype(str)

# Encode target
y = OrdinalEncoder().fit_transform(y.values.reshape(-1, 1)).ravel()

# Define columns
numeric_features = ['V1', 'V2', 'V3', 'V4']
categorical_features = ['V5']

# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    ('cat', OrdinalEncoder(), categorical_features)
])

# Pipeline with preprocessing and RFE
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('rfe', RFE(estimator=LogisticRegression(), n_features_to_select=2))
])

# Fit the pipeline
pipeline.fit(X, y)

# Get feature mask from RFE
support_mask = pipeline.named_steps['rfe'].support_

# Combine feature names after preprocessing
feature_names = numeric_features + categorical_features

# Print selected top 2 features
top_features = [name for name, selected in zip(feature_names, support_mask) if selected]
print("Top 2 features selected by RFE:", top_features)


Top 2 features selected by RFE: ['V1', 'V3']


In [96]:
feature_names

['V1', 'V2', 'V3', 'V4', 'V5']

In [97]:
support_mask

array([ True, False,  True, False, False])

In [99]:
from sklearn.feature_selection import SequentialFeatureSelector

In [104]:
X_preprocessed = preprocessor.fit_transform(X)

# Apply SFS with LogisticRegression
sfs = SequentialFeatureSelector(LogisticRegression(), n_features_to_select=2, direction='backward')
sfs.fit(X_preprocessed, y)

# Get selected indices
selected_indices = list(sfs.get_support(indices=True))

In [105]:
selected_indices

[2, 3]

In [106]:
print("Indices of top 2 features selected by SFS (forward):", selected_indices)
# then V2, V4 th feature

Indices of top 2 features selected by SFS (forward): [2, 3]
